In [253]:

import pandas as pd


eventos = pd.read_csv('logs_entrenamiento.csv')

notas = pd.read_csv('notas.csv')

chapter = pd.read_csv('map_displayname_to_chap_seq_id.csv')

column_names_df = notas.columns


column_names_eventos = eventos.columns

# Print the column names
print("Column names of 'df':", column_names_df)
print("Column names of 'eventos':", column_names_eventos)

# Rename a single column
notas.rename(columns={'Username': 'username'}, inplace=True)

notas = notas.replace('Not Attempted', 0)

print(notas)


Column names of 'df': Index(['Unnamed: 0', 'Username', 'Grade', 'Grade Scaled',
       'Quiz 1: Uso de modelos matemáticos en la economía',
       'Quiz 2: Motivación y Función de Producción',
       'Quiz 3: Preferencias y Toma de Decisiones',
       'Quiz 4: Salario y Efectos',
       'Quiz 5: Explicando Diferencias y Conclusión',
       'Quiz 6: Introducción y conceptos iniciales',
       'Quiz 7: Prediciendo el resultado de un juego',
       'Quiz 8: Evaluación de Resultados',
       'Quiz 9: Determinación de Asignaciones', 'Quiz 10: Desigualdad',
       'Quiz 11: Conceptos centrales', 'Quiz 12: Profundizando lo aprendido',
       'Quiz 13: Conceptos centrales', 'Quiz 14: Profundizando lo aprendido',
       'Quiz 15: Conceptos centrales', 'Quiz 16: Profundizando lo aprendido',
       'Quiz 17: Introducción',
       'Quiz 18: Oferta, demanda y equilibrio de mercado',
       'Quiz 19: Cambios en la oferta y la demanda',
       'Quiz 20: Excedentes y la mano invisible',
       'Quiz 2

In [254]:
#QUIZES DE LAS SEQ

# Group the data by 'chap_order' and print unique values of 'display_name' for each group
grouped = chapter.groupby('seq_order')['display_name'].unique()

for chap_order, unique_display_names in grouped.items():
    print(f'Group {chap_order}: {unique_display_names}')


Group 0: ['Quiz 2: Motivación y Función de Producción'
 'Quiz 6: Introducción y conceptos iniciales'
 'Quiz 8: Evaluación de Resultados' 'Quiz 17: Introducción'
 'Quiz 25: Regla de la mayoría']
Group 1: ['Quiz 1: Uso de modelos matemáticos en la economía'
 'Quiz 3: Preferencias y Toma de Decisiones'
 'Quiz 7: Prediciendo el resultado de un juego'
 'Quiz 9: Determinación de Asignaciones' 'Quiz 11: Conceptos centrales'
 'Quiz 13: Conceptos centrales' 'Quiz 15: Conceptos centrales'
 'Quiz 18: Oferta, demanda y equilibrio de mercado'
 'Quiz 22: Eficiencia y externalidades'
 'Quiz 26: La paradoja de Condorcet' 'Quiz 28: Decisiones intertemporales']
Group 2: ['Quiz 4: Salario y Efectos' 'Quiz 10: Desigualdad'
 'Quiz 12: Profundizando lo aprendido'
 'Quiz 14: Profundizando lo aprendido'
 'Quiz 16: Profundizando lo aprendido'
 'Quiz 19: Cambios en la oferta y la demanda'
 'Quiz 23: Corrección de fallas' 'Quiz 27: Teorema del votante mediano'
 'Quiz 29: Tópicos Adicionales']
Group 3: ['Quiz 5: 

In [255]:
# ... (Previous code for seq_1)

# Convert the columns to numerical data type
columns_to_convert_seq_0 = ['Quiz 2: Motivación y Función de Producción',
                            'Quiz 6: Introducción y conceptos iniciales',
                            'Quiz 8: Evaluación de Resultados', 
                            'Quiz 17: Introducción',
                            'Quiz 25: Regla de la mayoría']

notas[columns_to_convert_seq_0] = notas[columns_to_convert_seq_0].apply(pd.to_numeric, errors='coerce')

# Calculate the average for the columns
notas['seq_0_avg'] = notas[columns_to_convert_seq_0].mean(axis=1)

# Drop the original columns
notas.drop(columns=columns_to_convert_seq_0, inplace=True)

# Repeat the above steps for other sets of columns
# For example, for seq_1
columns_to_convert_seq_1 = ['Quiz 1: Uso de modelos matemáticos en la economía',
                            'Quiz 3: Preferencias y Toma de Decisiones',
                            'Quiz 7: Prediciendo el resultado de un juego',
                            'Quiz 9: Determinación de Asignaciones',
                            'Quiz 11: Conceptos centrales',
                            'Quiz 13: Conceptos centrales',
                            'Quiz 15: Conceptos centrales',
                            'Quiz 18: Oferta, demanda y equilibrio de mercado',
                            'Quiz 22: Eficiencia y externalidades',
                            'Quiz 26: La paradoja de Condorcet',
                            'Quiz 28: Decisiones intertemporales']

notas[columns_to_convert_seq_1] = notas[columns_to_convert_seq_1].apply(pd.to_numeric, errors='coerce')

notas['seq_1_avg'] = notas[columns_to_convert_seq_1].mean(axis=1)
notas.drop(columns=columns_to_convert_seq_1, inplace=True)



# For seq_2
columns_to_convert_seq_2 = ['Quiz 4: Salario y Efectos', 'Quiz 10: Desigualdad',
                            'Quiz 12: Profundizando lo aprendido',
                            'Quiz 14: Profundizando lo aprendido',
                            'Quiz 16: Profundizando lo aprendido',
                            'Quiz 19: Cambios en la oferta y la demanda',
                            'Quiz 23: Corrección de fallas',
                            'Quiz 27: Teorema del votante mediano',
                            'Quiz 29: Tópicos Adicionales']

notas[columns_to_convert_seq_2] = notas[columns_to_convert_seq_2].apply(pd.to_numeric, errors='coerce')

notas['seq_2_avg'] = notas[columns_to_convert_seq_2].mean(axis=1)
notas.drop(columns=columns_to_convert_seq_2, inplace=True)

# For seq_3
columns_to_convert_seq_3 = ['Quiz 5: Explicando Diferencias y Conclusión',
                            'Quiz 20: Excedentes y la mano invisible',
                            'Quiz 24: Otras fuentes de ineficiencia']

notas[columns_to_convert_seq_3] = notas[columns_to_convert_seq_3].apply(pd.to_numeric, errors='coerce')

notas['seq_3_avg'] = notas[columns_to_convert_seq_3].mean(axis=1)
notas.drop(columns=columns_to_convert_seq_3, inplace=True)


notas.rename(columns={'Quiz 21: Impuestos y Subsidios': 'seq_4_avg'}, inplace=True)

print(notas.head(10))


   Unnamed: 0                                           username  Grade  \
0           3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1           4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
2           5  c5f32ddce577c3f4cce3bcdcb5c8e0e03933b5cc62fa18...   0.99   
3           6  81528304670b5be911641ecc2ba3a21195234d10c3ab43...   1.00   
4           7  4427f64ffb5a05abefc1aa6936fca72e2f8fc24ad5f11d...   0.91   
5           8  afca888290f061f966a5f4ab433b53ac73734e00bdfce6...   0.79   
6           9  95ee01a5a7e3c41023fc379d6acb0f47860f2149caacb1...   0.85   
7          10  61d11a14e95e598a883d48aea410c474ca6f419ee8a5b0...   0.88   
8          12  4f824a96d26c45a3e047fd26f1f164371e86b4cab53895...   0.80   
9          13  ef14776d7e8bc0b6ad12b59d9b12e2d742e962dfc415b0...   0.95   

   Grade Scaled seq_4_avg  Quiz (Avg)  seq_0_avg  seq_1_avg  seq_2_avg  \
0           6.6       1.0    0.905172        0.8   0.977273   0.944444   
1           7.0       1.0 

In [256]:
#ACÁ SE TOMA INFO DE LOG_TRAIN PARA SUPLIR LA DATA DE NOTAS


# Convert the 'time_column' to a datetime object
eventos['time'] = pd.to_datetime(eventos['time'])

# Create new columns for month, day, and time
eventos['month'] = eventos['time'].dt.month
eventos['day'] = eventos['time'].dt.day
eventos['hora'] = eventos['time'].dt.strftime('%H:%M:%S')




#----------------------------------

# DURACION DE SESIONES (una sesion es un día con registro de ingreso)

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username', 'month', 'day'])['time'].agg(['min', 'max']).reset_index()

#
grouped['time_difference'] = (grouped['max'] - grouped['min']).dt.total_seconds() / 3600

# Merge the 'time_difference' column from the 'grouped' DataFrame into the 'eventos' DataFrame
eventos = eventos.merge(grouped[['username', 'month', 'day', 'time_difference']], on=['username', 'month', 'day'], how='left')



#----------------------------------

#ESTADISTICOS DE DUACION DE SESIONES

# Group the DataFrame by 'usuarioname' and calculate the average and standard deviation of 'time_difference'
grouped = eventos.groupby('username')['time_difference'].agg(['mean', 'std']).reset_index()

# Rename the columns for clarity
grouped.columns = ['username', 'duracion_sesion_(avg)', 'duracion_sesion_(std)']

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracion_sesion_(avg)', 'duracion_sesion_(std)']].drop_duplicates(), on='username', how='left')



#----------------------------------

#DURACION EN DÍAS DE SU USO DE EOL

# Extract the date part
eventos['date'] = eventos['time'].dt.date

# Group the DataFrame by 'usuario', 'month', and 'day' and calculate the min and max time
grouped = eventos.groupby(['username'])['date'].agg(['min', 'max']).reset_index()

# Calculate the time difference in hours
grouped['duracionEOL'] = (grouped['max'] - grouped['min']).dt.days

#Agregarlo a la data final
notas = notas.merge(grouped[['username', 'duracionEOL']].drop_duplicates(), on='username', how='left')


column_names = notas.columns.tolist()

print(column_names)

['Unnamed: 0', 'username', 'Grade', 'Grade Scaled', 'seq_4_avg', 'Quiz (Avg)', 'seq_0_avg', 'seq_1_avg', 'seq_2_avg', 'seq_3_avg', 'duracion_sesion_(avg)', 'duracion_sesion_(std)', 'duracionEOL']


In [257]:

#----------------------------------

#FRECUENCIA DE SESIONES POR MES


distinct_months = eventos['month'].unique()

# Create a new dataframe to store the results
result_data = {'username': eventos['username']}
for month in distinct_months:
    result_data[month] = eventos['username'].map(
        eventos[eventos['month'] == month].groupby('username')['day'].nunique()
    )

# Convert the dictionary to a DataFrame
result_df = pd.DataFrame(result_data)

# Rename multiple columns
result_df.rename(columns={8: 'Num Sesiones de Agosto', 9: 'Num Sesiones de Septiembre', 10: 'Num Sesiones de Noviembre'}, inplace=True)


result_df.fillna(0, inplace=True)

# Merge the 'result_df' with the 'df_notas' dataframe based on the 'username' column
notas = notas.merge(result_df, on='username', how='left')

# Print the updated 'eventos' DataFrame
print(notas)

column_names = notas.columns.tolist()

print(column_names)

        Unnamed: 0                                           username  Grade  \
0                3  05743498d061bf472df897f1e5bdda6d62e99312c952e3...   0.94   
1                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
2                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
3                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
4                4  2fb73a5b451fa1beba6a7e6c86cf66ab3d8bfa3fc256bb...   1.00   
...            ...                                                ...    ...   
536764         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536765         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536766         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536767         610  d4e09736a16c403937d6e829e6cdae6ed0537277c2c34d...   0.00   
536768         613  714d13c3517b88defa5763d78fc0c840a4bc1ad6092c02...   0.00   

        Grade Scaled seq_4_avg  Quiz (A

In [258]:
# Get unique values from a specific column
unique_values = eventos['grouped_event_type'].unique()
print(unique_values)

['/courses/(course-code)/course/'
 'edx.bi.course.upgrade.sidebarupsell.displayed'
 '/courses/(course-code)/progress' 'edx.ui.lms.link_clicked'
 '/courses/(course-code)/(uuid-course)/'
 '/courses/(course-code)/course_wiki'
 '/courses/(course-code)/wiki/(course-code)/'
 '/courses/(course-code)/discussion/(discussion-id)/'
 '/courses/(course-code)/jump_to/(block-code)@vertical@(uuid)'
 '/courses/(course-code)/courseware/(uuid1)/(uuid2)/(tab-n)'
 '/courses/(course-code)/xblock/block-v1:(block-code)@html+block@(uuid)/handler/publish_completion'
 'edx.ui.lms.sequence.next_selected'
 '/courses/(course-code)/courseware/(uuid-code1)/(uuid-code2)/(some-path)'
 'load_video'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/get_completion'
 '/courses/(course-code)/xblock/block-v1:(block-code)@sequential+block@(uuid)/handler/xmodule_handler/goto_position'
 'seq_goto' 'play_video' 'pause_video' 'seek_video' 'page_close'
 '/courses/(course-code)/xb

In [259]:
#SE ENCUENTRAN LOS ESTADISTICOS DE LAS ACTIVIDADES QUE REALIZA EN CADA SESION
# IE CANTIDAD DE VECES QUE REALIZA ALGUNA ACTIVIDAD EN UNA SESION (MONTH, DAY)
# List of event types to create columns for
event_types = ['page_close', 'problem_graded', 'problem_check', 'problem_show', 'seg_prev',
               'seg_next', 'seg_goto', 'load_video', 'play_video', 'pause_video', 'speed_change_video']

# Iterate through the event types and create columns with boolean values
for event_type in event_types:
    eventos[event_type] = (eventos['grouped_event_type'] == event_type).astype(int)

grouped_eventos = eventos.groupby(['username', 'month', 'day'])[event_types].sum().reset_index()



# Group the sub_df DataFrame by 'username' and calculate the mean and standard deviation for each event type
agg_df = grouped_eventos.groupby('username').agg({
    'page_close': ['mean', 'std'],
    'problem_graded': ['mean', 'std'],
    'problem_check': ['mean', 'std'],
    'problem_show': ['mean', 'std'],
    'seg_prev': ['mean', 'std'],
    'seg_next': ['mean', 'std'],
    'seg_goto': ['mean', 'std'],
    'load_video': ['mean', 'std'],
    'play_video': ['mean', 'std'],
    'pause_video': ['mean', 'std'],
    'speed_change_video': ['mean', 'std']
}).reset_index()


# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(agg_df, on='username', how='left')

# Display the resulting 'notas' DataFrame
column_names = notas.columns.tolist()

print(column_names)

['Unnamed: 0', 'username', 'Grade', 'Grade Scaled', 'seq_4_avg', 'Quiz (Avg)', 'seq_0_avg', 'seq_1_avg', 'seq_2_avg', 'seq_3_avg', 'duracion_sesion_(avg)', 'duracion_sesion_(std)', 'duracionEOL', 'Num Sesiones de Agosto', 'Num Sesiones de Septiembre', 'Num Sesiones de Noviembre', ('page_close', 'mean'), ('page_close', 'std'), ('problem_graded', 'mean'), ('problem_graded', 'std'), ('problem_check', 'mean'), ('problem_check', 'std'), ('problem_show', 'mean'), ('problem_show', 'std'), ('seg_prev', 'mean'), ('seg_prev', 'std'), ('seg_next', 'mean'), ('seg_next', 'std'), ('seg_goto', 'mean'), ('seg_goto', 'std'), ('load_video', 'mean'), ('load_video', 'std'), ('play_video', 'mean'), ('play_video', 'std'), ('pause_video', 'mean'), ('pause_video', 'std'), ('speed_change_video', 'mean'), ('speed_change_video', 'std')]


/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(
/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [260]:
#COUNT ACTIVIDADES PER CHAPTER (encontrar chapters que haya estudioado mas/menos)

grouped = eventos.groupby(['username', 'sequential'])['grouped_event_type'].count().reset_index()

pivoted_df = grouped.pivot(index='username', columns='sequential', values='grouped_event_type').fillna(0)

# Reset the index to make 'column1' a regular column
pivoted_df.reset_index(inplace=True)

# Rename columns for clarity
pivoted_df.columns.name = None

# Assuming 'notas' is the DataFrame where you want to add the columns
notas = notas.merge(pivoted_df, on='username', how='left')

# Rename multiple columns
notas.rename(columns={'0': 'num_eventos_seq_0', '1': 'num_eventos_seq_1', '2': 'num_eventos_seq_2', '3': 'num_eventos_seq_3', '4': 'num_eventos_seq_4'}, inplace=True)

# Display the resulting 'notas' DataFrame
column_names = notas.columns.tolist()

print(column_names)

['Unnamed: 0', 'username', 'Grade', 'Grade Scaled', 'seq_4_avg', 'Quiz (Avg)', 'seq_0_avg', 'seq_1_avg', 'seq_2_avg', 'seq_3_avg', 'duracion_sesion_(avg)', 'duracion_sesion_(std)', 'duracionEOL', 'Num Sesiones de Agosto', 'Num Sesiones de Septiembre', 'Num Sesiones de Noviembre', ('page_close', 'mean'), ('page_close', 'std'), ('problem_graded', 'mean'), ('problem_graded', 'std'), ('problem_check', 'mean'), ('problem_check', 'std'), ('problem_show', 'mean'), ('problem_show', 'std'), ('seg_prev', 'mean'), ('seg_prev', 'std'), ('seg_next', 'mean'), ('seg_next', 'std'), ('seg_goto', 'mean'), ('seg_goto', 'std'), ('load_video', 'mean'), ('load_video', 'std'), ('play_video', 'mean'), ('play_video', 'std'), ('pause_video', 'mean'), ('pause_video', 'std'), ('speed_change_video', 'mean'), ('speed_change_video', 'std'), 'num_eventos_seq_0', 'num_eventos_seq_1', 'num_eventos_seq_2', 'num_eventos_seq_3', 'num_eventos_seq_4', 'none_page']


In [261]:
#AGREGAR LABELS

label_df = eventos[['username', 'label']].drop_duplicates(subset='username')


merged_df = pd.merge(notas, label_df, on='username', how='inner')

merged_df.name = notas

/var/folders/x0/z6r3qq012s1ggpdmg3ps4ltr0000gn/T/ipykernel_23260/2118169417.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  merged_df.name = notas


In [262]:
#CREAMOS UN DICCIONARIO CON PARA PODER GUARDAR A LOS USUARIOS CON NOMBRES MAS PEQUEÑOS
import pandas as pd


# Genera nombres de usuario acortados automáticamente
notas['user_id'] = ['u' + str(i+1) for i in range(len(notas))]

# Crea un diccionario para mapear nombres acortados a nombres largos
diccionario_mapeo = dict(zip(notas['user_id'], notas['username']))

notas = notas.drop('username', axis=1)

column_order = ['user_id'] + [col for col in notas if col != 'user_id']
notas = notas[column_order]


#PARA ENCONTRAR A UN USUARIO SE USA LO SIGUIENTE
#nombre_acortado = 'u2'
#nombre_largo = diccionario_mapeo[nombre_acortado]

#print(f"Nombre largo correspondiente a '{nombre_acortado}': {nombre_largo}")


/Users/antoniaarias/opt/anaconda3/lib/python3.9/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


In [263]:
notas.sample(50)

,user_id,Unnamed: 0,Grade,Grade Scaled,seq_4_avg,Quiz (Avg),seq_0_avg,seq_1_avg,seq_2_avg,seq_3_avg,...,"(pause_video, mean)","(pause_video, std)","(speed_change_video, mean)","(speed_change_video, std)",num_eventos_seq_0,num_eventos_seq_1,num_eventos_seq_2,num_eventos_seq_3,num_eventos_seq_4,none_page
180586,u180587,186,0.96,6.7,1.0,0.931034,0.6,1.000000,1.000000,1.000000,...,8.529412,11.429773,0.352941,0.701888,210.0,426.0,355.0,27.0,14.0,344.0
433032,u433033,471,0.89,6.2,1.0,0.862069,0.8,0.863636,0.833333,1.000000,...,28.105263,39.811912,0.842105,1.572795,269.0,1321.0,633.0,73.0,12.0,412.0
333526,u333527,346,1.00,7.0,1.0,0.965517,0.8,1.000000,1.000000,1.000000,...,10.750000,13.679669,1.625000,1.784190,112.0,402.0,213.0,37.0,14.0,334.0
268098,u268099,274,0.94,6.6,1.0,0.911034,0.8,0.856364,1.000000,1.000000,...,10.210526,11.433766,1.631579,1.891811,127.0,811.0,339.0,23.0,2.0,347.0
43174,u43175,52,0.85,5.9,1.0,0.818966,0.8,0.886364,0.666667,1.000000,...,14.176471,19.956939,0.000000,0.000000,101.0,454.0,246.0,41.0,3.0,264.0
194084,u194085,205,1.00,7.0,1.0,0.965517,0.8,1.000000,1.000000,1.000000,...,6.588235,11.837751,0.176471,0.528594,122.0,525.0,199.0,52.0,33.0,391.0
128460,u128461,136,0.96,6.7,1.0,0.931034,0.8,1.000000,0.888889,1.000000,...,88.647059,64.292244,0.235294,0.970143,404.0,1959.0,967.0,240.0,4.0,235.0
512477,u512478,560,0.65,4.4,1.0,0.586207,0.4,0.575455,0.630000,0.666667,...,4.333333,9.154754,0.000000,0.000000,114.0,381.0,95.0,16.0,15.0,364.0
160648,u160649,165,0.81,5.6,1.0,0.781724,0.6,0.772727,0.796667,1.000000,...,4.785714,6.681531,1.928571,2.973325,131.0,353.0,171.0,50.0,17.0,285.0
306654,u306655,310,0.85,5.9,1.0,0.818966,0.8,0.795455,0.888889,0.666667,...,10.285714,9.848300,0.785714,1.050902,118.0,536.0,295.0,19.0,11.0,295.0


In [264]:
# Get all column names
column_names = notas.columns.tolist()

print(column_names)

['user_id', 'Unnamed: 0', 'Grade', 'Grade Scaled', 'seq_4_avg', 'Quiz (Avg)', 'seq_0_avg', 'seq_1_avg', 'seq_2_avg', 'seq_3_avg', 'duracion_sesion_(avg)', 'duracion_sesion_(std)', 'duracionEOL', 'Num Sesiones de Agosto', 'Num Sesiones de Septiembre', 'Num Sesiones de Noviembre', ('page_close', 'mean'), ('page_close', 'std'), ('problem_graded', 'mean'), ('problem_graded', 'std'), ('problem_check', 'mean'), ('problem_check', 'std'), ('problem_show', 'mean'), ('problem_show', 'std'), ('seg_prev', 'mean'), ('seg_prev', 'std'), ('seg_next', 'mean'), ('seg_next', 'std'), ('seg_goto', 'mean'), ('seg_goto', 'std'), ('load_video', 'mean'), ('load_video', 'std'), ('play_video', 'mean'), ('play_video', 'std'), ('pause_video', 'mean'), ('pause_video', 'std'), ('speed_change_video', 'mean'), ('speed_change_video', 'std'), 'num_eventos_seq_0', 'num_eventos_seq_1', 'num_eventos_seq_2', 'num_eventos_seq_3', 'num_eventos_seq_4', 'none_page']


In [265]:
notas.to_csv('data_para_entrenar.csv')

#PENDIENTE

- encontrar forma de incluir las variables de event type (conocer los distintos tipos de event type y sacar la frecuencia de cada uno en las sesiones y a partir de esto sacar estadisticos, ej en promedio cuantas veces pone pausa, a que vwlocidad ve ek video, repite mucho los mismos videos etc)
-quizas los estadísticos de la duracion de las sesiones podría ser especifico a cada mes
